In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from my_nlp_module.preprocessing import preprocess_document, PrepOption

import numpy as np
import pandas as pd
from gensim.models import KeyedVectors
import nltk
import re
from my_nlp_module.tokenizer import Tokenizer

path_to_model = "../pretrained_models/40/model.bin"
model = KeyedVectors.load_word2vec_format(path_to_model, binary=True)
embed_dim = model.vector_size

In [ ]:
def mean_doc_vector(document, embed_dim, embed_matrix):
    result = np.zeros(embed_dim,)
    for token in document:
        if token != 0:
            result += embed_matrix[token]
    result /= len(document)
    return result

In [3]:
texts = ["""I really like visiting museums and exploring culture. 
I want to visit some cultural events. I prefer cheap accomodation.""",
        """I love going to the night clubs. I hope to go to parties in the evenings. During the day I want to fine dine
        and visit some bars."""]

templates = ["museum history buildings culture tradition", "party nigh club clubs drinking alcohol", 
             "restaurant food expensive tasty", "cheap accomodation hostel"]

#options = [PrepOption.STOPWORDS, PrepOption.LOWERCASE, PrepOption.LEMMA, 
#           PrepOption.NUMBERS, PrepOption.RUBBISH, PrepOption.INTERPUNCTION]
options = [PrepOption.NUMBERS, PrepOption.RUBBISH, PrepOption.INTERPUNCTION]

documents_preprocessed = []

for doc in texts:
    p = preprocess_document(doc, options)
    documents_preprocessed.append(p)
        
df_train = pd.DataFrame({"document": documents_preprocessed})
df_templates = pd.DataFrame({"document": templates})

df_combined = pd.DataFrame({"document": templates+documents_preprocessed})


TypeError: preprocess_document() got an unexpected keyword argument 'language'

In [ ]:
MAX_WORDS = 10000
tok = Tokenizer(MAX_WORDS)
tok.fit(list(df_combined['document']))
vocab_size = len(tok.vocab) + 1

embed_matrix=np.zeros(shape=(vocab_size,embed_dim))
for word,i in tok.vocab.items():
    try:
        embed_vector=model[word]
        embed_matrix[i]=embed_vector
    except KeyError:
        continue

print(f"number of words {vocab_size}")

In [ ]:
encoded_texts = tok.texts_to_sequences(list(df_train['document']))
encoded_templates = tok.texts_to_sequences(list(df_templates['document']))

In [ ]:
seq_vectors_texts = np.asarray([mean_doc_vector(encoded_texts[i], embed_dim, embed_matrix) \
                                for i in range(len(encoded_texts))])
seq_vectors_templates = np.asarray([mean_doc_vector(encoded_templates[i], embed_dim, embed_matrix) \
                                for i in range(len(encoded_templates))])
print(f"Kształt wektorów treningowych: {seq_vectors_texts.shape}\n"\
     f"Kształt wektorów testowych: {seq_vectors_templates.shape}")

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

results = cosine_similarity(seq_vectors_texts, seq_vectors_templates)

rows = []
columns = ["numer tekstu", "muzea itd.", "imprezy", "restauracje", "tani nocleg"]
for i in range(results.shape[0]):
    row = [i]
    for j in range(results.shape[1]):
        row.append(results[i, j])
    rows.append(row)

df = pd.DataFrame(rows, columns=columns)
df
# czym bardziej wartość bliższa 1 tym bardziej tekst jest podobny do danej kategorii